In [2]:
import numpy as np
import pandas as pd
import Utils
from smt.sampling_methods import LHS
import pickle
from smt.problems import Branin
from scipy.optimize import minimize
from scipy.optimize import Bounds
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, Matern
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import rcParams
from matplotlib.transforms import offset_copy
import random
from collections import namedtuple
plt.style.use('ggplot')

## Bayesian Optimization

In [3]:
def bayesian_optimisation(n_iters, bounds, x0, alpha = 1e-5, epsilon = 1e-7, t0 = 2, tf = 1e-3):
    
    """ bayesian_optimisation
    Uses Gaussian Processes 
    Arguments:
    ----------
        n_iters: integer.
            Number of iterations to run the search algorithm.
        bounds: array-like, shape = [n_params, 2].
            Lower and upper bounds on the parameters of the function.
        x0: array-like, shape = [n_pre_samples, n_params].
            Array of initial points to sample the objective function. 
        alpha: double.
            Variance of the error term of the GP.
        epsilon: double.
            Precision tolerance for floats.
    """
    t = t0
    alp = np.power(tf / t0, 1/n_iters)
    f_x = np.zeros(n_iters) 
    
    n_params = 2
    x_list = []
    y_list = []
    
    for params in x0:
        x_list.append(params)
        y_list.append(Utils.branin(params))
        
    xp = np.array(x_list)
    yp = np.array(y_list)

    
    for n in range(n_iters):
        
        scaler = MinMaxScaler().fit(xp)
        
        kernel = Matern(nu = 3/2) 
        model = GaussianProcessRegressor(kernel = kernel, alpha = alpha, n_restarts_optimizer = 15, normalize_y = True, random_state = 0)
        model.fit(scaler.transform(xp),yp) #fit the model
        
        opt = Utils.find_robust_optimum (Model = model, Scaler = scaler) #find reference robust optimum
        
        #loc = Utils.find_robust_optimum_location (Model = model, Scaler = scaler)
        #noise = Utils.noise(Point = loc , Model = model, Scaler = scaler)
        #f_x[n] = Utils.branin(loc.reshape(1,-1) + noise.reshape(1,-1))
        f_x[n] = opt
        
        next_sample = Utils.sample_next_point (acquisition_func = Utils.MGFI, 
                                               Model = model, robust_optimum = opt,  temp = t, Scaler = scaler) #find next sample point by maximizing acquisiotn function

        
        if np.any(np.abs(next_sample - xp) <= epsilon):
            #print ('Iteration----'+str(n+1)+': Duplicate Sample Point')
            continue
        
        
        score = Utils.branin (next_sample)

        # Update lists
        x_list.append(next_sample.ravel())
        y_list.append(score)
        
        # Update xp and yp
        xp = np.array(x_list)
        yp = np.array(y_list)

        #update temperature
        t = alp * t
        
    return xp, yp, opt, model, f_x

## 1. Set the Global Parameters

In [4]:
n = 20
n_total =  120
dimensionality = 2
bounds = ((-4.25,9.25),(0.75,14.25))
xlimits = np.array([[-5,10],[0,15]])
print ('Global Parameters Set..............')

Global Parameters Set..............


## 2. Choose Initial Samples and run Bayesian Optimization

In [ ]:
runs = 25
res = []
for run in range(runs):
    print (run + 1)
    X = Utils.DOE(n_obs = n , xlimits = xlimits, random_state = run + 1 , criterion = 'm' )
    Temp = bayesian_optimisation( n_iters = n_total-n , bounds = bounds, x0 = X, t0 = 3.2371127309367878, tf = 0.1)
    res.append(Temp)
pickle.dump(res, open("res.p", "wb"))

1
2
3
4
5
6
